In [ ]:
import pandas as pd
import re
import os
import cx_Oracle
import datetime
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.externals import joblib
from nltk.stem.snowball import SnowballStemmer
from PIL import Image
from wordcloud import (WordCloud, get_single_color_func)
import matplotlib.pyplot as plt

stemmer=SnowballStemmer("russian")  

stopwords=[u'и',u'в',u'во',u'что',u'он',u'на',u'я',u'с',u'со',u'как',u'а',u'то',u'все',u'она',u'так',u'его',u'но',u'да',u'ты',u'к',u'у',u'же',
u'за',u'бы',u'по',u'только',u'ее',u'мне',u'было',u'вот',u'от',u'меня',u'еще', u'о',u'из',u'ему',u'теперь',u'когда',u'даже',u'ну',u'вдруг',
u'ли',u'если',u'уже',u'или',u'ни',u'быть',u'был',u'него',u'до',u'вас',u'нибудь',u'опять',u'уж',u'вам',u'ведь',u'там',u'потом',u'себя',u'ничего',
u'ей',u'может',u'они',u'тут',u'где',u'есть',u'надо',u'ней',u'для',u'мы',u'тебя',u'их',u'чем',u'была',u'сам',u'чтоб',u'без',u'будто',u'чего',u'раз',
u'тоже',u'себе',u'под',u'будет',u'ж',u'тогда',u'кто',u'этот',u'того',u'потому',u'этого',u'какой',u'совсем',u'ним',u'здесь',u'этом',u'один',u'почти',u'мой',
u'тем',u'чтобы', u'rur', u'rub', u'ru', u'руб', u'рублей', u'ндс', u'ooo', u'oao', u'без', u'ао', u'зао', u'г', u'ы', u'пао', u'гуп', u'ук',
u'кпк', u'кх', u'бст', u'хк', u'ук', u'январь', u'февраль', u'март',u'апрель',u'май', u'июнь', u'июль', u'август', u'сентябрь', u'октябрь', u'ноябрь', u'декабрь',
u'июл', u'феврал', u'распоряжение', u'облагается', u'n',  u'шт', u'период', u'г', u'eur', u'маы', u'апрел', u'июн', u'январ', u'иуне',
u'иуле', u'дог', u'рублях', u'начисление', u'экз',  u'sd', u'возврат', u'взимание', u'ед', u'е', u'юр', u'а',u'б',u'в',u'г',u'д',u'е',u'ё',
u'ж',u'з',u'и',u'к',u'л',u'м',u'н',u'о',u'п',u'р',u'с',u'т',u'у',u'ф',u'х',u'ц',u'ч',u'ш',u'щ',u'ъ',u'ы',u'ь',u'э',u'ю',u'я', u'р', u'б', u'н', u'нал', u'сч',
u'запрос', u'ип', u'монеты', u'дс', u'бум', u'лоро', u'заявке', u'нальчик', 'a','b','v','g','d','e','e','zh','z','i','k','l','m','n','o','p','r','s','t','u','f','h',
u'вал', u' подразд', u'приема', u'см', u'usd', u'sek', u'nok',  u'орому', u'орым', u'lc', u'usd', u'ов', u'vo', u'осб', u'сзб', u'visa', u'mastercard', u'тс',
u'ммвб', u'фб', u'дату', u'инн', u'кпп', u'ко', u'пс', u'контр', u'кампания', u'бух', u'ген', u'дир', u'кампании', u'через', u'над', u'ксб', u'скб', u'над',
u'списание', u'согл', u'распоряжению', u'свыше', u'виза', u'дог', u'файл', u'spe', u'spx', u'spdz', u'января',u'февраля', u'марта', u'апреля', u'мая', u'июня', u'июля', u'августа', u'сентября',
u'октября', u'ноября', u'декабря', u'зао', u'ооо', u'оао', u'ао', u'без', u'пао', u'гуп', u'ук', u'кпк', u'кх', u'бст', u'хк']

In [84]:
def create_new_directory(name):
    if not os.path.exists(name):
            os.mkdir(name)
            
def delete_punctuation(s):
    return ' '.join((re.sub(r'[№"\'-_/.:?!1234567890()%<>;,+#$&\s+]', u' ', s)).split())

def delete_stopwords(s):
    return ' '.join([word for word in (re.sub(r'[()\s+]', u' ', s)).split() if word not in stopwords])

def get_campaign():
    start_time=datetime.datetime.now()
    con=cx_Oracle.connect('iskra/iskraurskb@iskra1_cskostat')
    query=  "select distinct pl_id_k_in_crm from( "\
            "select/*+ RESULT_CACHE*/ "\
                "case when pl_id_k_in_crm like '%ЦА%ЗП%ФОТ%' then "\
                "pl_id_k_in_crm || ' (ПИРС)' "\
             "else pl_id_k_in_crm end pl_id_k_in_crm, "\
             "pl_num_id, date_load, pl_status, pl_brand_com, pl_main_chanal "\
             "from atb_segmen_kp_pl)"
    print(query)
    data=pd.read_sql(query, con=con)
    print("End extracting data: "+ str(datetime.datetime.now()-start_time))
    return data

def get_data(campaign_name):
    start_time=datetime.datetime.now()
    con=cx_Oracle.connect('iskra/iskraurskb@iskra1_cskostat')
    query="select REGEXP_REPLACE(ltrim(REGEXP_REPLACE(TEXT, '[[:digit:]|[:punct:]]+', \' \'), \' \'), \'( ){2,}\', \' \') TEXT "\
          "from ( select  /*+ parallel(32)*/ lower(TO_CHAR(SUBSTR(tasks_comment,0, DBMS_LOB.getlength(tasks_comment)))) TEXT "\
          "from atb_segmen_tasks_kamp where status = 'Закрыта' and tasks_comment is not null and date_load='15.12.2017' and CAMPAIGN_ID_ZADACHA like '%"+campaign_name+"%')"
    print(query)
    data=pd.read_sql(query, con=con)
    print("End extracting data: "+ str(datetime.datetime.now()-start_time))
    return data

def get_stat(data, filename):
    df=pd.DataFrame()
    for cls in list(data['class'].drop_duplicates()):
        data_split=data[data['class']==cls]
        df=df.append([[cls, data_split['class'].count()]])
    df.columns=['class', 'count']
    df.to_csv(filename+'_stat.csv', sep=';')
    
def cut_words_freq2(words_freq):
    # для того, чтобы свернуть похожие ключевые слова
    keys=list(words_freq.keys())
    for word in keys:
        if word in words_freq.keys():
            stemmed_word=stemmer.stem(word) 
            similar_keys=[s for s in words_freq.keys() if stemmed_word in s]

            similar_word_freq={}
            for similar_key in similar_keys:
                similar_word_freq[similar_key]=words_freq[similar_key]   
            if len(similar_word_freq)!=0:
                word_max=max(similar_word_freq, key=similar_word_freq.get)

                for key in similar_keys:
                    if word_max !=key:
                        words_freq[word_max]+=words_freq[key]
                        words_freq.pop(key)
                        
    return words_freq
    
def cut_words_freq(phrases):
    keys=list(phrases.keys())
    for phrase in keys:
        if phrase in phrases.keys():
            phrase=stemming(phrase.split())
            print(phrase)

            similar_phrases_freq={}
            for word in phrase.split():
                similar_keys=[s for s in phrases.keys() if word in s]
                print(similar_keys)
                for similar_key in similar_keys:
                    print(phrases)
                    if similar_key=='':
                        continue
                    similar_phrases_freq[similar_key]=phrases[similar_key]   

                    if len(similar_phrases_freq)!=0:
                        phrase_max=max(similar_phrases_freq, key=similar_phrases_freq.get)

                        for i in range(len(similar_keys)):
                             if word_max !=similar_keys[i]:
                                print(similar_keys[i])
                                phrases[word_max]+=phrases[similar_keys[i]]
                                phrases.pop(similar_keys[i])
                                similar_keys[i]=''
    return phrases

def get_cloud(count_words, data, filename, flag=0):
    cloud_mask=np.array(Image.open("cloud form.png"))
    fig=plt.figure(figsize=(45,30))
    wordcloud=WordCloud(collocations=False, background_color="white", mask=cloud_mask, stopwords=stopwords, max_words=count_words)   
    wordcloud.generate_from_frequencies(frequencies=data)
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.axis("off")
    plt.show()      
    fig.savefig(filename+'.png') 
    if flag==1:
        return wordcloud
       
def collect(x):
    for s in x.split(' '):
        if (len(s)<4) & (s not in collection) & (s not in ['нет', 'не', 'др', 'уфк', 'физ']):
            collection.append(s)
            
def get_colors():
    colors=list(matplotlib.colors.cnames.keys())
    colors=colors[::2] 
    len(colors)
    return colors

def get_color_big_class():
    classes=pd.read_excel('Все кластеры.xlsx', encoding='cp1251')
    hierach_classes={}
    for cls, big_cls in zip(classes['Кластер - 2'], classes['Кластер - 1']):
        hierach_classes[cls]=big_cls
    neg_pos_color_big_cls={'Встреча': 'green',
                           'Выполнено': 'green',
                           'Неудачный контакт': 'grey',
                           'Отказ': 'red',
                           'Ошибка задачи': 'grey',
                           'Перспектива': 'green',
                           'Проблемный клиент': 'red',
                           'Условия': 'red'}
    
    neg_pos_color_small_cls={}
    for key in neg_pos_color_big_cls.keys():
        if neg_pos_color_big_cls[key] in neg_pos_color_small_cls.keys():
            neg_pos_color_small_cls[neg_pos_color_big_cls[key]]=neg_pos_color_small_cls[neg_pos_color_big_cls[key]]+(list(classes[classes['Кластер - 1']==key]['Кластер - 2'].values))
        else:
            neg_pos_color_small_cls[neg_pos_color_big_cls[key]]=list(classes[classes['Кластер - 1']==key]['Кластер - 2'].values)
    print(neg_pos_color_small_cls)  
    
    small_cls={}
    for key in neg_pos_color_small_cls.keys():
        for values in neg_pos_color_small_cls[key]:
            small_cls[values]=key
    return small_cls

def valid_date(string):
    string=string.split(' ')
    parsed_string=''
    for token in string:
            try:
                 parsed_string+=' '+(re.sub(r'(\d{2}).(\d{2}).?(\d{0,4})', r'дата\время', token))
            except ValueError:
                 parsed_string+=' '+token
    return parsed_string


def stemming(words):
    stemmed_words=[]
    for word in words:
        stemmed_words.append(stemmer.stem(word))
    return ' '.join(stemmed_words)

def preprocessing(data):
    collection=[]
    data=data.dropna()
    data['TEXT'].apply(lambda x: collect(x))
    stopwords=stopwords+collection
    data['TEXT']=data['TEXT'].apply(lambda x: delete_stopwords(x))
    data['TEXT']=data['TEXT'].apply(lambda x: delete_punctuation(valid_date(x.lower())).split())    
    print(data['TEXT'])
    data['TEXT']=data['TEXT'].apply(lambda x: stemming(x))
    
    data['TEXT']=data['TEXT'].apply(lambda x: x.replace('не ', 'не_').replace('нет ', 'нет_').replace('др ', 'др_'))
    return data

In [ ]:

class SimpleGroupedColorFunc(object):
    """Create a color function object which assigns EXACT colors
       to certain words based on the color to words mapping
       Parameters
       ----------
       color_to_words : dict(str -> list(str))
         A dictionary that maps a color to the list of words.
       default_color : str
         Color that will be assigned to a word that's not a member
         of any value from color_to_words.
    """

    def __init__(self, color_to_words, default_color):
        self.word_to_color = {word: color
                              for (color, words) in color_to_words.items()
                              for word in words}

        self.default_color = default_color

    def __call__(self, word, **kwargs):
        return self.word_to_color.get(word, self.default_color)


class GroupedColorFunc(object):
    """Create a color function object which assigns DIFFERENT SHADES of
       specified colors to certain words based on the color to words mapping.
       Uses wordcloud.get_single_color_func
       Parameters
       ----------
       color_to_words : dict(str -> list(str))
         A dictionary that maps a color to the list of words.
       default_color : str
         Color that will be assigned to a word that's not a member
         of any value from color_to_words.
    """

    def __init__(self, color_to_words, default_color):
        self.color_func_to_words = [
            (get_single_color_func(color), set(words))
            for (color, words) in color_to_words.items()]

        self.default_color_func = get_single_color_func(default_color)

    def get_color_func(self, word):
        """Returns a single_color_func associated with the word"""
        try:
            color_func = next(
                color_func for (color_func, words) in self.color_func_to_words
                if word in words)
        except StopIteration:
            color_func = self.default_color_func

        return color_func

    def __call__(self, word, **kwargs):
        return self.get_color_func(word)(word, **kwargs)

In [ ]:
classes=pd.read_excel('Все кластеры.xlsx', encoding='cp1251')
hierach_classes={}
for cls, big_cls in zip(classes['Кластер - 2'], classes['Кластер - 1']):
    hierach_classes[cls]=big_cls
hierach_classes

In [ ]:
#load
digit_class=joblib.load('declasses_dict.pkl')
classifier=joblib.load('clf_done.pkl')

#Получение списка компаний
# campaigns=get_campaign()
# campaigns=pd.read_excel('count компаний продаж.xlsx', encoding='cp1251')
# campaigns=campaigns['CAMPAIGN_ID_ZADACHA'].values[:10]
# campaigns=campaigns.dropna()
# print(campaigns.shape[0])
# campaigns=campaigns['PL_ID_K_IN_CRM'].values
campaigns=['ЦА-ПРОДММБ-0117' ]
for campaign in campaigns:
    collection=[]
    create_new_directory(campaign)
    data=get_data(campaign)
    print(campaign +': '+str(data.shape[0]))
    data=data.dropna()
    data['TEXT'].apply(lambda x: collect(x))
    stopwords=stopwords+collection
    if 'нет' in stopwords:
        break
    data['TEXT']=data['TEXT'].apply(lambda x: delete_stopwords(x))
    data['TEXT']=data['TEXT'].apply(lambda x: x.replace('не ', 'не_').replace('нет ', 'нет_').replace('др ', 'др_'))
    data['class']= classifier.predict(data['TEXT'].values)
    data['class']=data['class'].apply(lambda x: digit_class[x])
    data=data[data['class']!='мусор']
    data.to_csv(campaign+'\\'+campaign+'.csv', sep=';')
    get_stat(data, campaign+'\\'+campaign)

    #Отрисовка облаков
      
    group_words={}
    for cls in list(data['class'].drop_duplicates()):
        data_split=data[data['class']==cls]
        
        vectorizer=CountVectorizer(max_features=30, ngram_range=(3,3))
        counts = vectorizer.fit_transform(data_split['TEXT']).toarray().sum(axis=0)                                               
        words = vectorizer.get_feature_names()    
        print(cls)
        print(words)
        
        words_freq={}
        for word, count in zip(words, counts):
            words_freq[word]=count
        words_freq=cut_words_freq(words_freq)            
        group_words[cls]=words_freq
       
        create_new_directory(campaign+'\\clouds\\')
        get_cloud(10, words_freq, campaign+'\\clouds\\'+cls.replace('\\', '.'))
        
    all_words={}
    for lst in list(group_words.values()):
        all_words.update(lst)

    cloud=get_cloud(200, all_words, 'all', 1)

    class_colors=get_color_big_class()
    colored_words={}
    keys=list(group_words.keys())
    for key in keys:
        if key=='мусор':
            continue
        if class_colors[key] in colored_words:
            colored_words[class_colors[key]]= colored_words[class_colors[key]]+list(group_words[key])
        else:
            colored_words[class_colors[key]]=list(group_words[key])

            default_color='grey'
    grouped_color_func = GroupedColorFunc(colored_words, default_color)
    cloud.recolor(color_func=grouped_color_func)
    fig=plt.figure(figsize=(45,30))
    plt.imshow(cloud, interpolation="bilinear")
    plt.axis("off")
    plt.show()

    fig.savefig(campaign+'\\clouds\\all_colored.png')   
        

In [ ]:
campaigns=['ЦА-ПРОДММБ-0117']
for campaign in campaigns:
    create_new_directory(campaign)
    data=get_data(campaign)
    data=data.dropna()
    
    #stopwords
    collection=[]
    data['TEXT'].apply(lambda x: collect(x))
    stopwords=stopwords+collection
    
    
    data['TEXT']=data['TEXT'].apply(lambda x: delete_stopwords(x))
    data['TEXT']=data['TEXT'].apply(lambda x: delete_punctuation(valid_date(x.lower())).split())    
    print(data['TEXT'])
    data['TEXT_STEMMED']=data['TEXT'].apply(lambda x: stemming(x))
    data['TEXT']=data['TEXT'].apply(lambda x: ' '.join(x))
    print(campaign +': '+str(data.shape[0]))
    data.to_csv('preprocessing_result.csv', sep=';', encoding='cp1251')
   

In [10]:
#load
digit_class=joblib.load('declasses_dict.pkl')
classifier=joblib.load('clf_done.pkl')

data['class']= classifier.predict(data['TEXT'].values)
data['class']=data['class'].apply(lambda x: digit_class[x])
data=data[data['class']!='мусор']
data.to_csv(campaign+'\\'+campaign+'.csv', sep=';')


In [ ]:
#Отрисовка облаков
      
group_words={}
for cls in list(data['class'].drop_duplicates()):
    data_split=data[data['class']==cls]

    vectorizer=CountVectorizer(max_features=10, ngram_range=(2,2))
    counts = vectorizer.fit_transform(data_split['TEXT']).toarray().sum(axis=0)                                               
    words = vectorizer.get_feature_names() 
    print(cls)
    print(words)
    
    words_freq={}
    for word, count in zip(words, counts):
        words_freq[word]=count
    words_freq=cut_words_freq(words_freq)            
    group_words[cls]=words_freq
    
all_words={}
for lst in list(group_words.values()):
    all_words.update(lst)

cloud=get_cloud(100, all_words, 'all', 1)

class_colors=get_color_big_class()
colored_words={}
keys=list(group_words.keys())
for key in keys:
    if key=='мусор':
        continue
    if class_colors[key] in colored_words:
        colored_words[class_colors[key]]= colored_words[class_colors[key]]+list(group_words[key])
    else:
        colored_words[class_colors[key]]=list(group_words[key])

        default_color='grey'
# grouped_color_func = GroupedColorFunc(colored_words, default_color)
# cloud.recolor(color_func=grouped_color_func)
# fig=plt.figure(figsize=(45,30))
# plt.imshow(cloud, interpolation="bilinear")
# plt.axis("off")
# plt.show()

# fig.savefig(campaign+'\\clouds\\all_colored.png')   
        